# Advanced Data Structures — AVL Tree Lab

This notebook implements an AVL Tree in Python, demonstrates insertions and deletions, prints traversals and balance factors, and visualizes the tree.

Follow the cells below to run tests and generate the visualizations required for the lab.

## AVL Tree Implementation
The following cell implements an AVL tree with insert, delete, rotations, and traversal helpers. It also provides utilities for printing balance factors and drawing the tree.

In [ ]:
# AVL Tree implementation
class AVLNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def __init__(self):
        self.root = None
        self.rotation_log = []

    def height_of(self, node):
        return node.height if node else 0

    def update_height(self, node):
        node.height = 1 + max(self.height_of(node.left), self.height_of(node.right))

    def get_balance(self, node):
        if not node:
            return 0
        return self.height_of(node.left) - self.height_of(node.right)

    def _left_rotate(self, z):
        y = z.right
        T2 = y.left
        # rotation
        y.left = z
        z.right = T2
        # update heights
        self.update_height(z)
        self.update_height(y)
        self.rotation_log.append(f'Left rotation at node {z.key}')
        return y

    def _right_rotate(self, z):
        y = z.left
        T3 = y.right
        # rotation
        y.right = z
        z.left = T3
        # update heights
        self.update_height(z)
        self.update_height(y)
        self.rotation_log.append(f'Right rotation at node {z.key}')
        return y

    def _insert(self, node, key):
        # Standard BST insert
        if not node:
            return AVLNode(key)
        elif key < node.key:
            node.left = self._insert(node.left, key)
        else:
            node.right = self._insert(node.right, key)

        # Update height and balance
        self.update_height(node)
        balance = self.get_balance(node)

        # Check rotations
        # Left Left
        if balance > 1 and key < node.left.key:
            return self._right_rotate(node)
        # Right Right
        if balance < -1 and key > node.right.key:
            return self._left_rotate(node)
        # Left Right
        if balance > 1 and key > node.left.key:
            node.left = self._left_rotate(node.left)
            self.rotation_log.append(f'Left-Right rotation at node {node.key}')
            return self._right_rotate(node)
        # Right Left
        if balance < -1 and key < node.right.key:
            node.right = self._right_rotate(node.right)
            self.rotation_log.append(f'Right-Left rotation at node {node.key}')
            return self._left_rotate(node)

        return node

    def insert(self, key):
        self.root = self._insert(self.root, key)

    def _min_value_node(self, node):
        current = node
        while current.left:
            current = current.left
        return current

    def _delete(self, node, key):
        if not node:
            return node
        elif key < node.key:
            node.left = self._delete(node.left, key)
        elif key > node.key:
            node.right = self._delete(node.right, key)
        else:
            # Node with only one child or no child
            if not node.left:
                temp = node.right
                node = None
                return temp
            elif not node.right:
                temp = node.left
                node = None
                return temp
            # Node with two children
            temp = self._min_value_node(node.right)
            node.key = temp.key
            node.right = self._delete(node.right, temp.key)

        # Update height
        self.update_height(node)
        balance = self.get_balance(node)

        # Balance the node if needed
        if balance > 1 and self.get_balance(node.left) >= 0:
            return self._right_rotate(node)
        if balance > 1 and self.get_balance(node.left) < 0:
            node.left = self._left_rotate(node.left)
            return self._right_rotate(node)
        if balance < -1 and self.get_balance(node.right) <= 0:
            return self._left_rotate(node)
        if balance < -1 and self.get_balance(node.right) > 0:
            node.right = self._right_rotate(node.right)
            return self._left_rotate(node)

        return node

    def delete(self, key):
        self.root = self._delete(self.root, key)

    # Traversals
    def _preorder(self, node, res):
        if not node: return
        res.append(node.key)
        self._preorder(node.left, res)
        self._preorder(node.right, res)

    def preorder(self):
        res = []
        self._preorder(self.root, res)
        return res

    def _inorder(self, node, res):
        if not node: return
        self._inorder(node.left, res)
        res.append(node.key)
        self._inorder(node.right, res)

    def inorder(self):
        res = []
        self._inorder(self.root, res)
        return res

    def _postorder(self, node, res):
        if not node: return
        self._postorder(node.left, res)
        self._postorder(node.right, res)
        res.append(node.key)

    def postorder(self):
        res = []
        self._postorder(self.root, res)
        return res

    def height(self):
        return self.height_of(self.root)

    def collect_nodes(self):
        # returns list of (key, balance) via inorder traversal
        res = []
        def _collect(n):
            if not n: return
            _collect(n.left)
            res.append((n.key, self.get_balance(n)))
            _collect(n.right)
        _collect(self.root)
        return res

    # Simple drawing utility to compute positions (inorder x index, depth y)
    def _compute_positions(self):
        positions = {}
        x = 0
        def _inorder(node, depth=0):
            nonlocal x
            if not node: return
            _inorder(node.left, depth+1)
            positions[node.key] = (x, -depth)
            x += 1
            _inorder(node.right, depth+1)
        _inorder(self.root)
        return positions

    def draw(self, filename=None):
        import matplotlib.pyplot as plt
        pos = self._compute_positions()
        if not pos:
            print('Tree is empty')
            return
        fig, ax = plt.subplots(figsize=(8, 4))
        # draw nodes and edges recursively
        def _draw_node(node):
            if not node: return
            x, y = pos[node.key]
            ax.text(x, y, str(node.key), ha='center', va='center', bbox=dict(boxstyle='circle', facecolor='white', edgecolor='black'))
            if node.left:
                xl, yl = pos[node.left.key]
                ax.plot([x, xl], [y, yl], 'k-')
                _draw_node(node.left)
            if node.right:
                xr, yr = pos[node.right.key]
                ax.plot([x, xr], [y, yr], 'k-')
                _draw_node(node.right)
        _draw_node(self.root)
        ax.axis('off')
        if filename:
            plt.savefig(filename, bbox_inches='tight')
            print(f'Drawn tree saved to {filename}')
        else:
            plt.show()

## Task 2: Testing the AVL Tree (Insertions)
Insert the keys: 30, 40, 50, 20, 10, 5, 35, 25. After each insertion we print preorder and current balance factors.

In [ ]:
# Test insertions and show preorder + balance factors after each insertion
keys_to_insert = [30, 40, 50, 20, 10, 5, 35, 25]
avl = AVLTree()
for k in keys_to_insert:
    avl.insert(k)
    print(f'After inserting {k}:')
    print('  Preorder:', avl.preorder())
    print('  Balance factors (inorder):', avl.collect_nodes())
    print('  Rotations so far:', avl.rotation_log)
    print('-'*40)

# Draw final tree
avl.draw(filename='avl_after_insertions.png')
print('AVL height after insertions:', avl.height())

## Task 3: AVL Tree Deletion
Delete nodes: 20, 30, 35. Print preorder and balances after deletions and save visualization.

In [ ]:
# Perform deletions
deletions = [20, 30, 35]
for d in deletions:
    avl.delete(d)
    print(f'After deleting {d}:')
    print('  Preorder:', avl.preorder())
    print('  Balance factors (inorder):', avl.collect_nodes())
    print('  Rotations log:', avl.rotation_log)
    print('-'*40)

avl.draw(filename='avl_after_deletions.png')
print('AVL height after deletions:', avl.height())

## Task 4: Analysis: Height Comparison and Complexity
Compare AVL tree height with an unbalanced BST (insert same keys in same order). Also provide time complexity notes.

In [ ]:
# Simple unbalanced BST for comparison
class BSTNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None

def bst_insert(root, key):
    if not root:
        return BSTNode(key)
    if key < root.key:
        root.left = bst_insert(root.left, key)
    else:
        root.right = bst_insert(root.right, key)
    return root

def bst_height(root):
    if not root: return 0
    return 1 + max(bst_height(root.left), bst_height(root.right))

# Build unbalanced BST using the initial insertion list
bst_root = None
for k in keys_to_insert:
    bst_root = bst_insert(bst_root, k)

print('AVL height:', avl.height())
print('Unbalanced BST height:', bst_height(bst_root))

print('
Time complexity (AVL):
 - Search: O(log n)
 - Insert: O(log n)
 - Delete: O(log n)
')
print('AVL trees keep height approximately <= 1.44*log2(n) — better than unbalanced BST in worst cases.')

### Deliverables
Files produced by this notebook:
- `makori_seth_AVL_Lab.ipynb` (this notebook)
- `avl_after_insertions.png` (visualization after insertions)
- `avl_after_deletions.png` (visualization after deletions)

You can run the notebook in Jupyter or Colab. If you're running locally, ensure `matplotlib` is installed.